# Pruebas con TFIDF de boletines de AMLO

Uso el archivo json que generé con [este script](./../WebScrapping/webscrapping-amlo.ipynb).

Calculo TFIDF usando un boletín como documento y todos los boletines de AMLO como corpus.

In [1]:
import pandas as pd
from collections import Counter
from itertools import chain
from functools import reduce
import re
import io
from math import log

In [2]:
data = pd.read_json('./../Boletines/AMLO/json/boletines.json', dtype="dict", encoding="utf-8")
boletines = data['boletines']
corpus = [boletin["contenido"] for boletin in boletines]

In [3]:
corpus_tokenizado = [re.findall('[a-zA-zñáéíóúüÁÉÍÓÚÜÑ]+', contenido.lower()) for contenido in corpus if len(contenido)>2]
# ch = chain(*corpus_tokenizado)
# todos_los_tokens = list(ch)
# df_corpus = pd.DataFrame(todos_los_tokens)
# df_corpus.to_csv('Data/pln_corpus_amlo.csv')

In [4]:
with io.open('TFIDF/stop-words-spanish.txt', mode='r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()
corpus_tokenizado = [[w for w in doc if not (w in stopwords)] for doc in corpus_tokenizado]
counter_boletines = [Counter(contenido) for contenido in corpus_tokenizado]

In [5]:
def bm25(x, k=25):
    return ((x + 1)*k)/(x+k)

def tf(documento, func=bm25):
    c = Counter(documento)
    return {
        termino : bm25(cuenta) for (termino, cuenta) in c.items()
    }

def idf(palabra, documentos):
    return log(
        len(documentos) / 
        (1+reduce(lambda cuenta, documento : cuenta + (1 if (palabra in documento) else 0), documentos, 0))
    )

def tfidf(tf : dict, idfs : dict):
    return {
            palabra: idfs[palabra]*tf_value
            for (palabra, tf_value) in tf.items()
    }

def obtener_keywords(tfidfs, num_keywords=5):
    keywords = []
    for tfidf_n in tfidfs:
        keywords_n = sorted(tfidf_n.items() ,  key=lambda x: x[1], reverse=True)[:num_keywords]
        keywords.append([keyword[0] for keyword in keywords_n])
    return keywords

In [6]:
cuenta_total_palabras = reduce((lambda x, y: x + y), counter_boletines)
palabras = [palabra for (palabra, cuenta) in cuenta_total_palabras.items()]

In [7]:
idfs = {
    palabra: idf(palabra, corpus_tokenizado) for palabra in palabras
}

In [8]:
tfs = [tf(corpus_tokenizado[i]) for i in [1, 23, 87]]

In [9]:
tfidfs = [
    {
        palabra: idfs[palabra]*tf
        for (palabra, tf) in tf_n.items()
    }
    for tf_n in tfs
]

In [10]:
keywords = obtener_keywords(tfidfs, 15)
keywords

[['catarina',
  'técnicos',
  'apodaca',
  'mesa',
  'santa',
  'tlc',
  'aeropuerto',
  'viable',
  'léon',
  'dosis',
  'alternativo',
  'sostienen',
  'hundimiento',
  'n',
  'océano'],
 ['moral',
  'populismo',
  'redes',
  'corral',
  'sucia',
  'ético',
  'chihuahua',
  'mensajes',
  'seguidores',
  'contracampaña',
  'imperativo',
  'dominando',
  'quisieron',
  'documental',
  'roberto']]

In [11]:
counter_total = reduce(lambda x, y : x+y, counter_boletines, Counter())

In [13]:
counter_total.most_common(30)

[('méxico', 1001),
 ('lópez', 654),
 ('obrador', 650),
 ('va', 466),
 ('gobierno', 447),
 ('país', 409),
 ('pueblo', 403),
 ('candidato', 395),
 ('dijo', 392),
 ('si', 373),
 ('vamos', 370),
 ('manuel', 306),
 ('andrés', 296),
 ('presidencia', 274),
 ('historia', 272),
 ('juntos', 257),
 ('corrupción', 255),
 ('comentó', 253),
 ('campaña', 249),
 ('mil', 238),
 ('haremos', 233),
 ('coalición', 232),
 ('van', 223),
 ('pregunta', 223),
 ('presidente', 219),
 ('expresó', 218),
 ('poder', 217),
 ('indicó', 209),
 ('ahora', 204),
 ('pesos', 203)]